In [48]:
import openai
import faiss
import numpy as np
import sqlite3
import os
import pandas as pd

# Set up your OpenAI API key
openai.api_key = 'sk-ko4mD0G5q5eQe7c_mzU2rd5jJnQbWxGCIThHBbZ_dnT3BlbkFJA8DjlC9WP2q3D2IMvaEKuQkWHQlPewnDjHbDr45NwA' 

# Step 1: Extract schema metadata (table names and column names) from the database
def extract_schema_from_db(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get all table names from the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    schema = {}
    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()

        schema[table_name] = {}
        for col in columns:
            # col[1] is the column name, col[2] is the type (like int, text, OBJECT.)
            schema[table_name][col[1]] = col[2]  # Store column name and its type

    conn.close()

    print(f"Extracted Schema: {schema}")
    return schema


# Step 2: Vectorize schema metadata and store it in FAISS
def create_vector_db_from_schema(schema):
    schema_embeddings = []
    column_descriptions = []

    for table, columns in schema.items():
        for column_name, column_type in columns.items():
            # Combine table, column, and type for vectorization
            description = f"Table {table}, Column {column_name}, Type {column_type}"
            column_descriptions.append(description)
            
            # Get embedding for the description
            embedding = openai.Embedding.create(input=description, model="text-embedding-ada-002")['data'][0]['embedding']
            schema_embeddings.append(embedding)

    # Convert to numpy array for FAISS
    schema_embeddings = np.array(schema_embeddings).astype('float32')

    # Initialize FAISS index for schema embeddings
    d = schema_embeddings.shape[1]  # Dimensionality of embeddings ROOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOM FOR CHAAAAAAAAAAAAAAAAAAANGE
    index = faiss.IndexFlatIP(d)  # L2 distance index ROOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOM FOR CHAAAAAAAAAAAAAAAAAAAAAAAAANGE
    index.add(schema_embeddings)

    # Save the FAISS index and column descriptions
    faiss.write_index(index, "schema_index.faiss")
    np.save("column_descriptions.npy", column_descriptions)

    print(f"Vectorized schema and saved to FAISS.")
    return index


# Step 3: Retrieve relevant columns and table names based on user query
def retrieve_relevant_schema(query, schema_index_file="schema_index.faiss", column_descriptions_file="column_descriptions.npy", top_n=10):
    # Load FAISS index and descriptions
    index = faiss.read_index(schema_index_file)
    column_descriptions = np.load(column_descriptions_file, allow_pickle=True)

    # Get embedding for the user query
    query_embedding = openai.Embedding.create(input=query, model="text-embedding-ada-002")['data'][0]['embedding']
    query_embedding = np.array(query_embedding).astype('float32').reshape(1, -1)

    # Search for most relevant schema columns
    D, I = index.search(query_embedding, len(column_descriptions))

    # Return top N most relevant columns
    relevant_columns = [(column_descriptions[i], D[0][idx]) for idx, i in enumerate(I[0][:top_n])]

    if not relevant_columns:
        print("No relevant columns found!")
    else:
        print(f"Relevant columns: {relevant_columns}")

    return relevant_columns


# Step 4: Use GPT-4 to generate SQL based on relevant schema columns and table
def generate_sql_prompt(query, relevant_columns):
    table_name = None
    for col, _ in relevant_columns:
        if "Table" in col:
            table_name = col.split(",")[0].replace("Table", "").strip()
            break

    # Extract relevant columns
    relevant_col_names = [col.split(",")[1].replace("Column", "").strip() for col, _ in relevant_columns if "Column" in col]

    # If no table name is found, assume a default
    table_name = table_name or "titanic"

    # Create the SQL generation prompt for GPT-4
    context = f"Relevant columns: {', '.join(relevant_col_names)}."
    input_text = f"Generate only a valid SQL query for the following question: {query}. Use the table '{table_name}' and the relevant columns. Do not include any explanation, only the SQL query."

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in SQL generation."},
            {"role": "user", "content": input_text}
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.5,
    )

    return response['choices'][0]['message']['content'].strip()


# Step 5: Extract SQL query from GPT-4 response
def extract_sql_from_response(response_text):
    try:
        start_idx = response_text.upper().index("SELECT")
        end_idx = response_text.index(";", start_idx) + 1  # Find the first semicolon after SELECT
        sql_query = response_text[start_idx:end_idx].strip()
        return sql_query
    except Exception as e:
        print(f"Error extracting SQL from response: {e}")
        return None


# Step 6: Execute the SQL query against the SQLite database
def execute_sql_query(db_file, query):
    if not os.path.exists(db_file):
        print(f"Database file not found: {db_file}")
        return None

    conn = sqlite3.connect(db_file)
    try:
        df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None
    finally:
        conn.close()


# Step 7: Interpret the SQL result using GPT-4 to answer the original user question
def interpret_results(results, user_query):
    if results is None or results.empty:
        return "No results to interpret."

    # Convert the results DataFrame to a string format for passing to GPT-4
    results_str = results.to_string(index=False)

    # Construct the input text to provide context for GPT-4 to analyze and answer the user query
    input_text = f"Given the following SQL query result:\n{results_str}\nBased on this data, answer the original user question: '{user_query}' as accurately and completely as possible. Also give a analytical  summary of the result at the end that is short but is more numerical in nature"

    # Call GPT-4 for interpreting the results and answering the original query
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in interpreting data and providing answers based on SQL results."},
            {"role": "user", "content": input_text}
        ],
        max_tokens=300,
        n=1,
        stop=None,
        temperature=0.5,
    )

    # Return the final answer based on the SQL data and the user's query
    return response['choices'][0]['message']['content'].strip()



# Step 8: Full pipeline to run the entire process
def run_pipeline(user_query, db_file):
    # Step 1: Extract schema from the database
    schema = extract_schema_from_db(db_file)

    # Step 2: Vectorize the schema and store in FAISS (only once)
    if not os.path.exists("schema_index.faiss"):
        create_vector_db_from_schema(schema)

    # Step 3: Retrieve relevant schema columns based on user query
    relevant_columns = retrieve_relevant_schema(user_query)
    print(f"Relevant schema columns: {relevant_columns}")

    # Step 4: Generate SQL query using GPT-4 based on the relevant schema
    sql_query = generate_sql_prompt(user_query, relevant_columns)
    print(f"Generated SQL Query: {sql_query}")

    # Step 5: Execute SQL query
    results = execute_sql_query(db_file, sql_query)
    print(f"SQL Query Results:\n{results}")

    # Step 6: Interpret results
    interpretation = interpret_results(results, user_query)
    print(f"Interpretation:\n{interpretation}")


# Example usage
user_query = "Find all passengers who survived and paid more than 50 dollars for their fare."
db_file = 'titanic.db'  # Path to your SQLite database

# Run the full pipeline
run_pipeline(user_query, db_file)

Extracted Schema: {'titanic': {'PassengerId': 'INTEGER', 'Survived': 'INTEGER', 'Pclass': 'INTEGER', 'Name': 'TEXT', 'Sex': 'TEXT', 'Age': 'REAL', 'SibSp': 'INTEGER', 'Parch': 'INTEGER', 'Ticket': 'TEXT', 'Fare': 'REAL', 'Cabin': 'TEXT', 'Embarked': 'TEXT'}}
Relevant columns: [('Fare: The fare paid for the ticket.', 0.35786408), ('Survived: Indicates whether the passenger survived (0 = No, 1 = Yes).', 0.38732126), ('Parch: Number of parents/children aboard the Titanic.', 0.3902487), ('Ticket: The ticket number of the passenger.', 0.4037013), ('Cabin: The cabin number where the passenger stayed.', 0.4108491), ('Age: Age of the passenger in years.', 0.418823), ('Pclass: Ticket class (1st, 2nd, 3rd).', 0.42880145), ('Name: The full name of the passenger.', 0.42984462), ('SibSp: Number of siblings/spouses aboard the Titanic.', 0.43792555), ('Sex: Gender of the passenger (male or female).', 0.44298968)]
Relevant schema columns: [('Fare: The fare paid for the ticket.', 0.35786408), ('Survive